In [1]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json

# ✅ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")

# ✅ 책 데이터 로드
with open("merged_books_filtered.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 책 요약을 임베딩 변환
book_summaries = [book["summary"] for book in books_data]
book_titles = [book["title"] for book in books_data]

# 🔹 임베딩 변환
book_embeddings = embedding_model.encode(book_summaries, convert_to_numpy=True)

# 🔹 L2 정규화 (코사인 유사도 적용)
faiss.normalize_L2(book_embeddings)  # 벡터 정규화

# 🔹 FAISS 인덱스 생성 (L2 정규화된 벡터 기반)
dimension = book_embeddings.shape[1]  # 벡터 차원 (768)
faiss_index = faiss.IndexFlatIP(dimension)  # 🚀 코사인 유사도 적용 (Inner Product)
faiss_index.add(book_embeddings)  # 벡터 추가

# ✅ FAISS 인덱스 저장
faiss.write_index(faiss_index, "book_faiss_cosine_index.bin")

print("✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.")


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.


In [19]:
def find_similar_books(user_story, top_k=5):
    """유저 입력 스토리 → 코사인 유사도 기반 책 검색"""
    # 🔹 유저 입력을 문장 임베딩 변환
    user_embedding = embedding_model.encode([user_story], convert_to_numpy=True)

    # 🔹 유저 입력 벡터도 L2 정규화
    faiss.normalize_L2(user_embedding)

    # 🔹 FAISS 검색 (코사인 유사도 기반)
    distances, indices = faiss_index.search(user_embedding, top_k)

    # 🔹 검색된 책 목록 출력
    recommended_books = []
    for i in range(top_k):
        idx = indices[0][i]
        title = book_titles[idx]
        summary = book_summaries[idx]
        description = books_data[idx]["description"]  # 📌 원본 설명 가져오기
        recommended_books.append(f"📖 **{title}**\n📌 {summary}\n")

    return "\n".join(recommended_books)

# ✅ 테스트: 유저가 입력한 스토리를 바탕으로 책 추천
user_story = "한 남자는 오랜만에 고향을 찾았다가 우연히 오래전 실종된 친구의 흔적을 발견한다. 그 사건은 이미 해결된 것으로 알려졌지만, 그는 석연치 않은 점들을 의심하며 진실을 파헤치기 시작한다. 조사할수록 주변 사람들의 반응은 미묘하게 변하고, 그가 알던 과거와 현실이 점점 어긋나기 시작한다. 결국 그는 충격적인 진실과 마주하게 되며, 자신이 믿어왔던 모든 것이 뒤바뀌는 경험을 하게 된다."
print(find_similar_books(user_story, top_k=5))


📖 **다잉 아이**
📌 기억의 일부를 잃은 한 남자는 사고 이후 기이한 사건에 휘말린다. 점차 밝혀지는 진실 속에서 그는 자신을 둘러싼 음모와 원한의 실체를 마주하게 되고, 주변 인물들은 하나둘씩 파멸해간다. 결국 그는 충격적인 결말을 맞이하며 자신이 겪어온 사건의 진상을 알게 된다.

📖 **꿈에서 온 그녀**
📌 태어나는 순간 어머니를 잃고 아버지의 냉대 속에서 성장한 함지훈은 늘 상처와 죄책감을 안고 살아간다. 어느 날 꿈에서 광기 어린 웃음을 짓는 한 여인을 만나게 되고, 이후 현실에서 그녀와 마주한 뒤 그의 삶은 예측할 수 없는 방향으로 흔들리기 시작한다. 계속해서 일어나는 비극 속에서 지훈은 믿음과 의심, 사랑과 상실 사이를 오가며 점점 더 깊은 혼란에 빠진다. 점차 밝혀지는 진실은 그에게 삶과 운명에 대한 근본적인 질문을 던진다.

📖 **시간 도둑**
📌 템푸스에서 균형자로 일하는 태민. 그는 관리부에서 정산 오류 메일을 받고, 미팅리스트에서 요즘 자주 마주치는 하진의 이름을 발견하고, 시계가 이상하게 움직이는 걸 발견하지만, 며칠 뒤, 하진은 뻐꾸기시계 바늘이 또 휙 지나가는 걸 발견한다. 친구 보영에게 말하지만 믿지 않고, 시간이 사라졌다는 것을 증명하려 하지만, 이유를 알아내지 못한 채 포기하고 만다. 그러던 중 하진은 우연히 태민을 만나게 되고, 둘은 급격히 가까워진다. 그런데 자신의 시간 도둑이 태민이라고? 잃어버린 하진의 시간은 과연 어떻게 돌려받을 수 있을까?

📖 **안녕, 그리고 안녕히**
📌 낯선 태국에서 깜짝 놀란 일을 통해 한 사람을 만난다. 그는 상황에 놀랐고 반응에 놀랐으며, 마지막 순간까지 진심을 의심하며 사랑할 수 있을까를 고민한다.

📖 **내가 현상수배범이라니**
📌 평범한 일상을 살아가던 중 현상 수배자 명단에 오른 것을 알게 된 주인공은 자신의 과거와 얽히게 되고, 그 과정에서 자신의 과거와 얽힌 비밀을 마주하게 된다. 이 책은 단순한 스릴러를 넘어, 현대 사회의 인간 관계와 도덕적 딜레마를 흥미롭게 탐구한다

In [20]:
import faiss
import json
import requests
from sentence_transformers import SentenceTransformer

# ✅ 1️⃣ FAISS 인덱스 로드
faiss_index = faiss.read_index("book_faiss_cosine_index.bin")

# ✅ 2️⃣ 책 데이터 로드
with open("merged_books_filtered.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

book_titles = [book["title"] for book in books_data]
book_summaries = [book["summary"] for book in books_data]

# ✅ 3️⃣ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")

# ✅ 4️⃣ 네이버 API 호출 함수
def search_naver_api(user_query):
    """네이버 API를 사용하여 책 검색"""
    client_id = "YOUR_NAVER_CLIENT_ID"
    client_secret = "YOUR_NAVER_CLIENT_SECRET"
    url = "https://openapi.naver.com/v1/search/book.json"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": user_query,
        "display": 5,  # 최대 5권 검색
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["items"]
    else:
        print(f"🚨 네이버 API 호출 실패: {response.status_code}")
        return []

# ✅ 5️⃣ FAISS 검색 함수
def find_similar_books(user_story, top_k=5):
    """유저 입력 스토리 → 코사인 유사도 기반 책 검색"""
    # 🔹 유저 입력을 문장 임베딩 변환
    user_embedding = embedding_model.encode([user_story], convert_to_numpy=True)
    faiss.normalize_L2(user_embedding)  # 벡터 정규화

    # 🔹 FAISS 검색
    distances, indices = faiss_index.search(user_embedding, top_k)

    # 🔹 검색된 책 목록 출력
    recommended_books = []
    for i in range(top_k):
        idx = indices[0][i]
        title = book_titles[idx]
        summary = book_summaries[idx]
        recommended_books.append(f"📖 **{title}**\n📌 {summary}\n")

    return recommended_books

# ✅ 6️⃣ FAISS + 네이버 API 하이브리드 검색 함수
def hybrid_search(user_story, top_k=5):
    """FAISS에서 먼저 검색하고, 부족하면 네이버 API 호출"""
    faiss_results = find_similar_books(user_story, top_k)

    # 🔹 검색 결과가 없거나, 유사한 책이 부족할 경우 → 네이버 API 검색
    if not faiss_results:
        print("⚠️ FAISS에서 유사한 책이 없음 → 네이버 API 검색 실행")
        naver_results = search_naver_api(user_story)

        if naver_results:
            print("✅ 네이버 API 검색 결과:")
            for book in naver_results:
                print(f"📖 **{book['title']}**\n📌 {book['description']}\n")

            # FAISS에 추가하기 (선택적)
            # add_to_faiss(naver_results)

        return naver_results
    else:
        return faiss_results

# ✅ 7️⃣ 테스트 실행
user_story = "한 남자는 오랜만에 고향을 찾았다가 우연히 오래전 실종된 친구의 흔적을 발견한다."
print(hybrid_search(user_story, top_k=3))


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


['📖 **열대**\n📌 한 소설가는 학창 시절 읽다 만 책을 떠올리며 그 작품을 다시 찾으려 한다. 어느 날 머리맡에 두고 잠들었던 책이 흔적도 없이 사라지고, 그는 방방곡곡을 헤매며 찾지만 아무런 단서도 발견하지 못한다. 시간이 흘러 우연히 참가한 ‘침묵 독서회’에서 그는 잃어버린 책과 똑같은 표지를 발견하게 되고, 이를 계기로 예상치 못한 기이한 모험이 시작된다.\n', '📖 **냉정과 열정사이 Rosso & Blu 세트**\n📌 헤어진 연인 아오이와 쥰세이의 이야기. 과거의 기억을 더듬는 여자, 과거의 그림자를 서성이는 남자의 이야기.\n', '📖 **수키와 니니**\n📌 스무 살을 맞이한 은재는 엄마의 사촌인 숙희 이모(수키)로부터 놀러 오라는 권유를 받고 먼 이국으로 향한다. 그곳에서 친구의 딸인 니니와 단둘이 살고 있던 20년 전, 은재는 시한부로 병을 앓는 수키가 자신을 찾는다는 말을 듣고 수키네로 향하지만, 20년 전, 니니에게는 어떤 일이 있었던 것일까. 20년 전, 니니는 백골 시체가 발견되었다. 20년 전, 니니에게는 어떤 일이 있었던 것일까. 20년 전, 니니에게는 어떤 일이 있었던 것일까. 20년 전, 니니에게는 어떤 일이 있었던 것일까. 20년 전, 니니에게는 어떤 일이 있었던 것일까. 20년 전, 니니에게는 어떤 일이 있었던 것일까. 20년 전, 니니에게는 어떤 일이 있었던 것일까. 20년 후 한국을 떠나고, 한 달 뒤 한국에 돌아오고 나서 소식이 끊긴 지 20년이 지나, 은재는 시한부로 병을 앓는 수키가 자신을 찾는다는 말을 듣고 오랜만에 수키네로 향하는 길. 수키가 건넨 것은 놀랍게도 니니의 실종과 관련된 기사 스크랩 북.\n']
